In [1]:
# imports
import pandas as pd
import numpy as np

import utility as u

# Acquire

* Data has been acquired from Kaggle at https://www.kaggle.com/datasets/kamilpytlak/personal-key-indicators-of-heart-disease
* The data set contained 319,795 observations and 18 features before cleaning
* Each observation represents one person in the study
* Each feature represents an aspect of each person's health

In [2]:
df = pd.read_csv('heart_2020_cleaned.csv')

# Prepare

* There were no null values in this data set
* Outliers for BMI and SleepTime will be removed using the inner quartile (k = 3)


In [12]:
df = df.rename(columns={'HeartDisease':'has_heart_disease', 
                        'BMI':'bmi', 
                        'Smoking':'is_smoker', 
                        'AlcoholDrinking':'heavy_drinker',
                        'Stroke':'stroke',
                        'PhysicalHealth':'poor_physical_health_days', 
                        'MentalHealth':'poor_mental_health_days', 
                        'DiffWalking': 'has_difficulty_walking', 
                        'Sex':'sex', 
                        'AgeCategory':'age_category',
                        'Race':'race', 
                        'Diabetic':'diabetic', 
                        'PhysicalActivity':'physical_activity', 
                        'GenHealth':'general_health', 
                        'SleepTime':'sleep_time',
                        'Asthma':'asthma', 
                        'KidneyDisease':'kidney_disease', 
                        'SkinCancer':'skin_cancer'})

In [13]:
df.columns

Index(['has_heart_disease', 'bmi', 'is_smoker', 'heavy_drinker', 'had_stroke',
       'poor_physical_health_days', 'poor_mental_health_days',
       'has_difficulty_walking', 'sex', 'age_category', 'race', 'diabetic',
       'physical_activity', 'general_health', 'sleep_time', 'asthma',
       'kidney_disease', 'skin_cancer', 'BMI_is_outlier',
       'SleepTime_is_outlier'],
      dtype='object')

In [4]:
u.get_outlier_cutoffs(df, ['BMI','SleepTime'])

BMI: Median = 27.34 upper_bound = 53.59 lower_bound = 1.8599999999999994
SleepTime: Median = 7.0 upper_bound = 14.0 lower_bound = 0.0


In [5]:
u.get_upper_and_lower_bound(df, ['BMI','SleepTime'])

(14.0, 0.0)

In [6]:
u.remove_outliers(df, ['BMI','SleepTime'])

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
1688,Yes,12.48,Yes,No,Yes,12.0,2.0,No,Male,50-54,Black,No,No,Fair,12.0,No,Yes,No
5037,No,12.75,Yes,Yes,No,0.0,0.0,No,Male,18-24,White,No,Yes,Excellent,5.0,No,No,No
6247,No,13.87,No,Yes,No,0.0,0.0,No,Male,70-74,White,No,No,Very good,8.0,No,No,No
8471,No,13.72,Yes,No,No,10.0,30.0,Yes,Female,75-79,White,No,No,Good,8.0,No,No,No
10356,No,13.31,Yes,No,No,1.0,15.0,Yes,Female,50-54,White,No,No,Fair,2.0,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292711,No,12.48,Yes,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,No
306969,No,13.35,No,No,No,10.0,7.0,No,Female,25-29,White,No,Yes,Good,5.0,No,No,No
309940,No,12.75,Yes,No,No,0.0,0.0,No,Female,80 or older,White,No,Yes,Fair,8.0,No,No,No
311320,No,13.38,No,No,No,30.0,0.0,Yes,Female,60-64,White,No,Yes,Good,9.0,Yes,No,Yes
